In [1]:
from openai import OpenAI
import urllib
from bs4 import BeautifulSoup
import requests
from config import Naver_config, Openai_config
from FlagEmbedding import BGEM3FlagModel
from embedding_utils import cosine_similarity  # https://github.com/openai/openai-cookbook/blob/main/examples/utils/embeddings_utils.py
import json
import numpy as np
client_id = Naver_config.client_id
client_secret = Naver_config.Client_secret
openai_client = OpenAI(api_key = Openai_config.API_key)

/opt/conda/envs/llama3/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def search_naver_blog(keyword):
    encText = urllib.parse.quote(keyword)
    url = "https://openapi.naver.com/v1/search/blog?query=" + encText
    query_response = urllib.request.Request(url)
    query_response.add_header("X-Naver-Client-Id",client_id)
    query_response.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(query_response)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        result = json.loads(response_body.decode('utf-8'))
    else:
        result = "fail"
    link_list = []
    desc_list = []
    for item in result.get('items'):
        if 'blog.naver.com' in item['link']:
            link_list.append(item['link'])
            desc_list.append(item['description'].replace('<b>','').replace('</b>',''))
    result = [item  for item in result.get('items') if 'blog.naver.com' in item['link'] ]
    return result, link_list, desc_list

In [3]:
def search_content(link):
    link = link.replace("blog","m.blog")
    # print(link)
    response = requests.get(link)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, "html.parser")
    title = soup.find('meta',property="og:title")['content']
    # print(title)
    content = []
    res_postct = soup.findAll('div',class_="post_ct")
    try:
        content = " ".join([item.get_text(strip=True) for item in res_postct]).split('URL복사신고하기')[1]
    except:
        content = " ".join([item.get_text(strip=True) for item in res_postct])
    return title, content

In [4]:
def bge_top_k(model, question, content_list, link_list, top_k, max_length):
    sentence_pairs = [[question,i] for i in content_list]
    scores = model.compute_score(sentence_pairs,max_passage_length=max_length,weights_for_different_modes=[0.5,0.1,0.3])

    dict_ = [{'content':content,'score':score } for content, score in zip(content_list, scores['colbert+sparse+dense'])]
    top_k_index = np.argsort(scores['colbert+sparse+dense'])[-top_k:].tolist()
    top_k_desc = np.array(content_list)[top_k_index].tolist()
    top_k_link = np.array(link_list)[top_k_index].tolist()
    return dict_, top_k_desc, top_k_link

In [5]:
def openai_top_k(emb_model, question, content_list, link_list, top_k, max_length):
    q_embed = emb_model.create(input = question, model='text-embedding-3-large').data[0].embedding
    c_embeds = emb_model.create(input = content_list, model='text-embedding-3-large')
    c_embeds = [data.embedding for data in c_embeds.data]
    scores = []
    for c_embed in c_embeds:
        score = cosine_similarity(q_embed, c_embed)
        scores.append(score)
    top_k_index = np.argsort(scores)[-top_k:].tolist()
    top_k_desc = np.array(content_list)[top_k_index].tolist()
    top_k_link = np.array(link_list)[top_k_index].tolist()
    dict_ = [{'content':content,'score':score } for content, score in zip(content_list, scores)]
    return dict_, top_k_desc, top_k_link

In [6]:
bge_model = BGEM3FlagModel("BAAI/bge-m3",use_fp16=True,device = "cuda")
openai_emb_model = openai_client.embeddings

Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 267721.53it/s]


In [7]:
question = 'M1 맥북 프로와 에어의 차이를 비교해줘. 각 모델의 성능을 중심으로 비교해줘.'
keyword = 'M1 맥북 프로 에어 비교'
result, link_list, desc_list = search_naver_blog(keyword)

In [8]:
bge_scores, bge_k_desc, bge_k_link = bge_top_k(bge_model, question, desc_list, link_list, len(desc_list), 200)
openai_scores, openai_k_desc, openai_k_link = openai_top_k(openai_emb_model, question, desc_list, link_list, len(desc_list), 200)

In [13]:
bge_scores

[{'content': '기존에 맥북 에어 M1을 구매해서 3년 정도 사용하고 있었다. 맥북 프로 인텔 노트북을 5년 정도... 디자인이 전작 인텔 맥북 프로와 비교하면 많이 바뀌었다. 터치바도 사라지고, 한때 밀고 있던 나비식... ',
  'score': 0.6691791415214539},
 {'content': '애플의 맥북 에어 m2, 프로13, m1을 대략적으로 비교해 봤는데요. 필자가 생각하는 결론은 m1의 가성비는 아직도 유효하다는 것! m2를 구매한다고 했을 때 필자는 프로를 구매할 것 같습니다.',
  'score': 0.7142210602760315},
 {'content': '또한 XDR Display의 최대 지속밝기는 1,000니트로 기존 M1 맥북의 최대 밝기(에어 400니트, 프로 500니트)와 비교했을 때 배 이상 좋아졌다고 볼 수 있습니다. 정말 쨍합니다. 제가 M1 맥북프로 제품을 1년 반정도... ',
  'score': 0.5746729969978333},
 {'content': '연식 비교 맥북 프로 16 M1 MAX 2021년 모델과 맥북 에어 15 M2 2023년식입니다. 두 화면에서 보듯이 상단에 제품 사진? 도 조금 달라 보입니다. 이번에는 액정 비교를 한번 해봤습니다. 사진을 찍는 제품은 니콘 z6에... ',
  'score': 0.6385824680328369},
 {'content': '그래서 비교해 본 맥북 에어 M2 13인치 vs. M1 맥북 프로 16인치입니다. 맥북 에어 M2 크기는? 2022년 새로운 모습으로 출시된 에어는 가로 30.41cm, 세로 21.5cm, 무게 1.24kg입니다. 우리가 흔히 접하는 잡지 크기와... ',
  'score': 0.6431470513343811},
 {'content': '16인치와 13인치 화면 크기 비교 현재는 M1을 넘어 성능 강화가 이루어진 M2, M3 칩셋까지 탑재된 맥북이... 아래는 맥북 에어 M1으로 FHD, 4K 영상 소스를 프리미어 프로, 파이

In [14]:
openai_scores

[{'content': '기존에 맥북 에어 M1을 구매해서 3년 정도 사용하고 있었다. 맥북 프로 인텔 노트북을 5년 정도... 디자인이 전작 인텔 맥북 프로와 비교하면 많이 바뀌었다. 터치바도 사라지고, 한때 밀고 있던 나비식... ',
  'score': 0.6200553318765225},
 {'content': '애플의 맥북 에어 m2, 프로13, m1을 대략적으로 비교해 봤는데요. 필자가 생각하는 결론은 m1의 가성비는 아직도 유효하다는 것! m2를 구매한다고 했을 때 필자는 프로를 구매할 것 같습니다.',
  'score': 0.7038716904822652},
 {'content': '또한 XDR Display의 최대 지속밝기는 1,000니트로 기존 M1 맥북의 최대 밝기(에어 400니트, 프로 500니트)와 비교했을 때 배 이상 좋아졌다고 볼 수 있습니다. 정말 쨍합니다. 제가 M1 맥북프로 제품을 1년 반정도... ',
  'score': 0.5010952771127822},
 {'content': '연식 비교 맥북 프로 16 M1 MAX 2021년 모델과 맥북 에어 15 M2 2023년식입니다. 두 화면에서 보듯이 상단에 제품 사진? 도 조금 달라 보입니다. 이번에는 액정 비교를 한번 해봤습니다. 사진을 찍는 제품은 니콘 z6에... ',
  'score': 0.6488671501908558},
 {'content': '그래서 비교해 본 맥북 에어 M2 13인치 vs. M1 맥북 프로 16인치입니다. 맥북 에어 M2 크기는? 2022년 새로운 모습으로 출시된 에어는 가로 30.41cm, 세로 21.5cm, 무게 1.24kg입니다. 우리가 흔히 접하는 잡지 크기와... ',
  'score': 0.6246612314989783},
 {'content': '16인치와 13인치 화면 크기 비교 현재는 M1을 넘어 성능 강화가 이루어진 M2, M3 칩셋까지 탑재된 맥북이... 아래는 맥북 에어 M1으로 FHD, 4K 영상 소스를 프리미어 프로, 파이

In [10]:
bge_k_desc[::-1]

['애플의 맥북 에어 m2, 프로13, m1을 대략적으로 비교해 봤는데요. 필자가 생각하는 결론은 m1의 가성비는 아직도 유효하다는 것! m2를 구매한다고 했을 때 필자는 프로를 구매할 것 같습니다.',
 '기존에 맥북 에어 M1을 구매해서 3년 정도 사용하고 있었다. 맥북 프로 인텔 노트북을 5년 정도... 디자인이 전작 인텔 맥북 프로와 비교하면 많이 바뀌었다. 터치바도 사라지고, 한때 밀고 있던 나비식... ',
 '16인치와 13인치 화면 크기 비교 현재는 M1을 넘어 성능 강화가 이루어진 M2, M3 칩셋까지 탑재된 맥북이... 아래는 맥북 에어 M1으로 FHD, 4K 영상 소스를 프리미어 프로, 파이널컷 영상 편집 시연을 다룬 포스팅인데... ',
 '그래서 비교해 본 맥북 에어 M2 13인치 vs. M1 맥북 프로 16인치입니다. 맥북 에어 M2 크기는? 2022년 새로운 모습으로 출시된 에어는 가로 30.41cm, 세로 21.5cm, 무게 1.24kg입니다. 우리가 흔히 접하는 잡지 크기와... ',
 '연식 비교 맥북 프로 16 M1 MAX 2021년 모델과 맥북 에어 15 M2 2023년식입니다. 두 화면에서 보듯이 상단에 제품 사진? 도 조금 달라 보입니다. 이번에는 액정 비교를 한번 해봤습니다. 사진을 찍는 제품은 니콘 z6에... ',
 '저희 사설 업체 애플모아에서는 M1 맥북 액정 교체로 프로 에어 두 제품 모두 당일 액정수리 가능한 곳입니다. 맥북 액정 불량 증상으로 위 사진속과 같이 깨져있는 화면이 확실하게 보일수도있구요. 반면에 화면... ',
 '또한 XDR Display의 최대 지속밝기는 1,000니트로 기존 M1 맥북의 최대 밝기(에어 400니트, 프로 500니트)와 비교했을 때 배 이상 좋아졌다고 볼 수 있습니다. 정말 쨍합니다. 제가 M1 맥북프로 제품을 1년 반정도... ']

In [11]:
openai_k_desc[::-1]

['애플의 맥북 에어 m2, 프로13, m1을 대략적으로 비교해 봤는데요. 필자가 생각하는 결론은 m1의 가성비는 아직도 유효하다는 것! m2를 구매한다고 했을 때 필자는 프로를 구매할 것 같습니다.',
 '연식 비교 맥북 프로 16 M1 MAX 2021년 모델과 맥북 에어 15 M2 2023년식입니다. 두 화면에서 보듯이 상단에 제품 사진? 도 조금 달라 보입니다. 이번에는 액정 비교를 한번 해봤습니다. 사진을 찍는 제품은 니콘 z6에... ',
 '그래서 비교해 본 맥북 에어 M2 13인치 vs. M1 맥북 프로 16인치입니다. 맥북 에어 M2 크기는? 2022년 새로운 모습으로 출시된 에어는 가로 30.41cm, 세로 21.5cm, 무게 1.24kg입니다. 우리가 흔히 접하는 잡지 크기와... ',
 '기존에 맥북 에어 M1을 구매해서 3년 정도 사용하고 있었다. 맥북 프로 인텔 노트북을 5년 정도... 디자인이 전작 인텔 맥북 프로와 비교하면 많이 바뀌었다. 터치바도 사라지고, 한때 밀고 있던 나비식... ',
 '16인치와 13인치 화면 크기 비교 현재는 M1을 넘어 성능 강화가 이루어진 M2, M3 칩셋까지 탑재된 맥북이... 아래는 맥북 에어 M1으로 FHD, 4K 영상 소스를 프리미어 프로, 파이널컷 영상 편집 시연을 다룬 포스팅인데... ',
 '또한 XDR Display의 최대 지속밝기는 1,000니트로 기존 M1 맥북의 최대 밝기(에어 400니트, 프로 500니트)와 비교했을 때 배 이상 좋아졌다고 볼 수 있습니다. 정말 쨍합니다. 제가 M1 맥북프로 제품을 1년 반정도... ',
 '저희 사설 업체 애플모아에서는 M1 맥북 액정 교체로 프로 에어 두 제품 모두 당일 액정수리 가능한 곳입니다. 맥북 액정 불량 증상으로 위 사진속과 같이 깨져있는 화면이 확실하게 보일수도있구요. 반면에 화면... ']